# 3D_HOUSES

## Import all libraries

In [37]:
from glob import glob
from typing import Tuple, List
import natsort
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab

import geopandas as gpd
import rasterio
import rioxarray as rxr
from shapely.geometry import Point, Polygon

%matplotlib inline

## Global Variables

In [4]:
address = "Sint-Pietersplein%209,Gent"

DSM_path = "/Volumes/Samsung T7/DTM/**/*.tif"
DTM_path =  "/Volumes/Samsung T7/DSM/**/*.tif"

shp_paths = ["/Volumes/Samsung T7/CADASTRE DATA/Belgium_L72_2020/Bpn_CaBu.shp", "/Volumes/Samsung T7/CADASTRE DATA/Belgium_L72_2020/Bpn_ReBu.shp"]

## Create a dictionary of GTiff's paths and bounds

In [9]:
def load_data(DSM_path:str, DTM_path:str) -> dict:
    
    data = {}
    data['DSM_list'] = natsort.natsorted([file for file in glob(DSM_path, recursive=True)])
    data['DTM_list'] = natsort.natsorted([file for file in glob(DTM_path, recursive=True)])
    data['bounds'] = []
    
    for path in data['DSM_list']:
        with rasterio.open(path, driver="GTiff") as tif:
            data['bounds'].append(np.array(tif.bounds))

    return data

In [10]:
data = load_data(DSM_path, DTM_path)

## Fetching the L_72 coordinates from the API with address

In [ ]:
def fetch_coord(address:str)  -> Point:
    pass

In [11]:
coord = Point(104994.91, 192612.04)

## Find the relevant GTiff file containing the coordinates

In [12]:
def find_tif(data:dict, coord:Point) -> Tuple[str]:
    x, y = coord.xy
    for tif_index, bounds in enumerate(data['bounds']):
        x_min, y_min, x_max, y_max = bounds
        if (x_min < x < x_max) and (y_min < y < y_max):
            return data["DSM_list"][tif_index], data["DSM_list"][tif_index]

In [13]:
tiffs = find_tif(data, coord)

## Find the polygon containing the coordinates

In [23]:
def find_polygon(coord:Point, shp_paths:List[str]):

    for path in shp_paths:
        try:
            poly = gpd.read_file(path, mask=coord).geometry[0]
            break
        except:
            continue
    return poly

In [25]:
poly = find_polygon(coord, shp_paths)

## Crop DSM & DTM

In [28]:
def crop_tif(tiffs:Tuple[str], poly, crop_method="bbox") -> Tuple:
        
    DSM_tif, DTM_tif = tiffs
    DSM = rxr.open_rasterio(DSM_tif,masked=True)
    DTM = rxr.open_rasterio(DTM_tif,masked=True)

    if crop_method == "bbox":
        left, bottom, right, top = poly.bounds
        DSM_clip = DSM.rio.clip_box(left, bottom, right, top)
        DTM_clip = DTM.rio.clip_box(left, bottom, right, top)

    elif crop_method == "points":
        DSM_clip = DSM.rio.clip(poly.__geo_interface__)
        DTM_clip = DTM.rio.clip(poly.__geo_interface__)

    else:
        raise ValueError("crop_method only takes values 'bbox' or 'points'")

    return DSM_clip, DTM_clip

In [32]:
DSM_clip, DTM_clip = crop_tif(tiffs, poly)

## Create CHM from DSM & DTM

In [30]:
def CHMer(DSM_clip, 
          DTM_clip):
    return DSM_clip - DTM_clip

In [3]:
CHM = CHMer(DSM_clip, DTM_clip)

NameError: name 'CHMer' is not defined

## 3D Rendering

In [1]:
def render_3D(CHM_clip):
    arr = CHM_clip.squeeze().data
    arr = np.pad(arr, [(2, ), (2, )], mode='constant')
    mlab.figure(size=(640, 800), bgcolor=(0.16, 0.28, 0.46))
    surf = mlab.surf(arr)
    mlab.zlabel("Height")
    mlab.show()

In [2]:
render_3D(CHM)

NameError: name 'CHM' is not defined